# Mixture-of-Agents in 46 lines of code

Reference to the Mixture of Experts (MoE) paper - https://github.com/togethercomputer/MoA


<img src="https://github.com/togethercomputer/MoA/raw/main/assets/moa-explained.png" height=400 width=600>

In [9]:
%pip install langchain-openai langchain-anthropic langchain-mistralai langchain getpass nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [1]:
import asyncio
import os
import getpass
from langchain_openai.chat_models import ChatOpenAI
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
from nest_asyncio import apply
apply()

In [6]:
# Set the API keys for the models
if os.environ.get('OPENAI_API_KEY') is None:
    os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")
if os.environ.get('ANTHROPIC_API_KEY') is None:
    os.environ['ANTHROPIC_API_KEY'] = getpass.getpass("Enter your Anthropic API key: ")
if os.environ.get('MISTRAL_API_KEY') is None:
    os.environ['MISTRAL_API_KEY'] = getpass.getpass("Enter your Mistral API key: ")

In [10]:
user_prompt = "What are 3 fun things to do in SF?"
reference_models = [
    ChatOpenAI(model='gpt-4o'),
    ChatAnthropic(model="claude-3-5-sonnet-20240620"),
    ChatMistralAI(),
]
aggregator_model = ChatOpenAI(model='gpt-4o')
aggreagator_system_prompt = """You have been provided with a set of responses from various open-source models to the latest user query. Your task is to synthesize these responses into a single, high-quality response. It is crucial to critically evaluate the information provided in these responses, recognizing that some of it may be biased or incorrect. Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.

Responses from models:"""

async def run_llm(model):
    """Run a single LLM call with a reference model."""
    for sleep_time in [1, 2, 4]:
        try:
            response = await model.ainvoke([HumanMessage(content=user_prompt)])
            return response.content
        except Exception as e:
            print(f"Error with {model.__class__.__name__}: {e}")
            await asyncio.sleep(sleep_time)
    return "Failed to get response"

San Francisco is a city rich in culture, history, and natural beauty. Here are three fun activities to consider during your visit:

1. **Explore Golden Gate Park**: This expansive urban park offers something for everyone. Key attractions include the California Academy of Sciences, which houses an aquarium, planetarium, and natural history museum. You can also visit the de Young Museum for art exhibits, the Japanese Tea Garden for a serene experience, and the San Francisco Botanical Garden to enjoy a diverse range of plant species. Renting a bike is a great way to cover more ground and take in the park's lush scenery.

2. **Visit Fisherman's Wharf and Pier 39**: This bustling waterfront area is a must-see for tourists. Enjoy fresh seafood at one of the many restaurants, watch sea lions lounging on the docks, and explore various shops and street performances. From here, you can also take a ferry to Alcatraz Island, where you can tour the infamous former prison and learn about its history

In [ ]:
async def main():
    results = await asyncio.gather(*[run_llm(model) for model in reference_models])

    final_response = await aggregator_model.ainvoke(
        [
            SystemMessage(
                content=aggreagator_system_prompt
                + "\n"
                + "\n".join(
                    [f"{i+1}. {str(element)}" for i, element in enumerate(results)]
                )
            ),
            HumanMessage(content=user_prompt),
        ]
    )

    print(final_response.content)


asyncio.run(main())